In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df=pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.optim import Adam

In [6]:
df["Experience"]=df["Experience"].apply(lambda x: x if x>0 else 0)

In [7]:
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [8]:
columns=['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Online', 'CreditCard','Personal Loan']
df=df[columns]

In [9]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [10]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.30,random_state=69)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(xtrain)
x_test=sc.transform(xtest)

In [12]:
x_train=torch.from_numpy(x_train).to(torch.float32)
y_train=torch.from_numpy(ytrain).to(torch.float32)
print(x_test)

[[ 1.07593396  1.19775453  1.21764639 ... -0.56372977  0.82087774
  -0.64087556]
 [ 0.1127859   1.02401224  1.04356545 ...  1.04172389  0.82087774
  -0.64087556]
 [ 1.35815228 -0.19218379 -0.17500107 ...  0.31636832  0.82087774
  -0.64087556]
 ...
 [ 0.92546374  0.06842964  0.08612032 ...  1.3125233   0.82087774
  -0.64087556]
 [ 0.59470663  1.02401224  0.95652499 ... -0.56372977 -1.2182082
  -0.64087556]
 [-0.95437383 -1.14776639 -1.21948667 ... -0.56372977  0.82087774
   1.56036532]]


In [13]:
bankdata=TensorDataset(x_train,y_train)
batch_size=16
dt = DataLoader(bankdata, batch_size=batch_size, shuffle=True)

In [14]:
class Model(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.hidden=nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid=nn.Sigmoid()
    def forward(self, x):
        x = self.layer1(x)
        x = self.hidden(x)
        x = self.layer2(x)
        x = self.sigmoid(x) 
        return x

In [15]:
input_size = x_train.shape[1]
hidden_size = 4
output_size = 1
mlp = Model(input_size, hidden_size, output_size)
mlp

Model(
  (layer1): Linear(in_features=11, out_features=4, bias=True)
  (hidden): ReLU()
  (layer2): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [16]:
loss_func = nn.BCELoss()
optimizer = Adam(mlp.parameters(), lr=0.01)

In [17]:
epoch = 20
training_loss = [0] * epoch
training_accuracy = [0] * epoch

for i in range(epoch):
    epochloss=0
    acc=0
    for x_batch, y_batch in dt:
        pred_y=mlp(x_batch)
        #print(list(mlp.parameters()))
        loss=loss_func(pred_y,y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epochloss+=loss.item()*x_batch.size(0)
        cnt=(torch.where(pred_y>=0.5,1,0)==y_batch.unsqueeze(1)).sum().float().item()
        acc+=cnt
    print(i,epochloss/len(dt.dataset),acc/len(dt.dataset))

0 0.24704529745238168 0.9142857142857143
1 0.10624416393893106 0.9642857142857143
2 0.08704255201268409 0.9688571428571429
3 0.08094226375967264 0.9714285714285714
4 0.07895749938275133 0.9711428571428572
5 0.07562257383577525 0.9708571428571429
6 0.07170280072518757 0.9728571428571429
7 0.0712399291430733 0.9751428571428571
8 0.0697365660209741 0.9734285714285714
9 0.0668432174999533 0.9748571428571429
10 0.06654070991730052 0.9757142857142858
11 0.06642246782513601 0.9774285714285714
12 0.06747315933555365 0.9725714285714285
13 0.06356283084183399 0.978
14 0.06409811498517437 0.9771428571428571
15 0.06383510290937765 0.9765714285714285
16 0.06274396895863382 0.9777142857142858
17 0.06214941064747317 0.9785714285714285
18 0.06310204505122134 0.9768571428571429
19 0.06310696487501263 0.9762857142857143


In [18]:
x_test=torch.from_numpy(x_test).to(torch.float32)
y_test=torch.from_numpy(ytest).to(torch.float32)

In [19]:
bankdata=TensorDataset(x_test,y_test)
batch_size=len(x_test)
data = DataLoader(bankdata, batch_size=batch_size, shuffle=True)

In [20]:
for x,y in data:
    pred=mlp(x)
    y=y.reshape(-1,1)
    cnt=(torch.where(pred>=0.5,1,0)==y).sum().float().item()
print(cnt/len(data.dataset))

0.978


In [21]:
from pyswarms.single import GlobalBestPSO

In [22]:
pop=np.concatenate([pa.detach().numpy().flatten() for pa in Model(input_size, hidden_size, output_size).parameters() ])
print(len(pop))

53


In [23]:
shape = [i.detach().numpy().shape for i in Model(input_size, hidden_size, output_size).parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

In [28]:
options = {'c1': 0.5, 'c2': 0.4, 'w': 0.56}
dim = len(pop)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
pso = GlobalBestPSO(n_particles=100, dimensions=53, options=options, bounds=bounds)

In [34]:
def acc(particle,shape=shape,size=size):
    accuracy = []
    out = []
    
    for p in particle:
        pa=list()
        c_sum=0
        for i in range(len(size)):
            array = p[c_sum : c_sum + size[i]]
            array = array.reshape(shape[i])
            c_sum += size[i]
            pa.append(array)
        pa = np.array(pa, dtype="object")
        out.append(pa)
    
    for i in range(len(out)):
        model = Model(input_size, hidden_size, output_size)
        for idx,wei in enumerate(model.parameters()):
            wei.data = (torch.tensor(out[i][idx])).to(torch.float32)
        
        pred_y = model(x_train)
        pred_y = torch.where(pred_y>=0.5, 1, 0).flatten()
        acc = (pred_y == y_train).sum().float().item() / 3500
        accuracy.append(1 - acc) 
        
    return accuracy

In [35]:
cost, parameters = pso.optimize(acc, iters=20)
print("Accuracy : ", 1 - best_cost)

2023-04-05 23:24:16,246 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.4, 'w': 0.56}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|20/20, best_cost=0.066
2023-04-05 23:24:20,116 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.06599999999999995, best pos: [-0.60635989  0.55519234  0.09892476  0.00804102  0.7125099  -0.56045421
  0.47664379 -0.01870445  0.30139904 -0.44121236  0.83649339 -0.05312667
  0.16132303 -0.32920519  0.88690478  0.16485606  0.32562499  0.48601781
  0.15644052  0.25002676  0.10812495 -0.00866497  0.8430342  -0.31987644
  0.06136458  0.42111701  0.189724    0.5300463   0.53880267 -0.99009501
  0.34949331  0.1550579   0.26965303  0.60191758 -0.02177436 -0.0678942
 -0.37206232  0.61803421  0.75838142 -0.37429806  0.81724363 -0.05890993
 -0.04814517  0.81532973  0.63904231  0.78796801 -0.03433053  0.10530848
 -0.01991945  0.36058121 -0.44789    -0.4717

Accuracy :  0.9334285714285714


In [58]:
result = []
for par in [parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = pa[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model(input_size, hidden_size, output_size)
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)

# Calculate Accuracy
y_pred = best_model(x_train)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == y_train).sum().float().item() / 3500
print("Accuracy : ", acc)

Accuracy :  0.934


In [59]:
from sklearn.metrics import classification_report
y_pred = best_model(x_test)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1438
           1       0.34      0.82      0.48        62

    accuracy                           0.93      1500
   macro avg       0.67      0.88      0.72      1500
weighted avg       0.97      0.93      0.94      1500

